# Lanzamiento de App

In [ ]:
# Intalación para lectura de token
! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=74205c8f227887085ab093fbe0c779ea8dbf94b31cfb56fd405cc20ea5fde9c2
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
# Token de autenticación en ngrok
! ngrok authtoken XXXX

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# ngrok para abrir un tunel desde la máquina virtual de Colab
# ngrok abre un tunel a su sevidor
# y permite conectarse al servidor dash que se ejecutará en la máquina virtual

import os.path
import sys, json
import requests
import subprocess

Tunnel= None
TunnelUrl= None

def tunnel_start():
  global Tunnel, TunnelUrl
  
  # Si no se tenía ngrok, se descarga
  if not os.path.isfile('ngrok'):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip -o ngrok-stable-linux-amd64.zip
  
  # Tunel activo, entonces desactivar
  if Tunnel and Tunnel.poll()!=None:
    Tunnel= None

  # Tunel no se estaba ejecutando
  if not Tunnel:
    Tunnel= subprocess.Popen(["./ngrok","http","8050"]) # Lanzamiento de proceso
  
  
  tunnel_res= requests.get("http://localhost:4040/api/tunnels")
  tunnel_str= tunnel_res.text
  tunnel_cfg= json.loads(tunnel_str)

  TunnelUrl= tunnel_cfg['tunnels'][0]['public_url']
  print("Podes acceder desde "+TunnelUrl)
  return TunnelUrl  

In [ ]:
# Aplicacion Dash

%%writefile my_dash_app1.py
import dash
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output # Clases Input y Output

# Plotly
import plotly.express as px

import pandas as pd
import numpy as np
import requests
import zipfile

# Carga de datos
# Csv
with zipfile.ZipFile("denue_inegi_56_clean.zip", "r") as zip_ref:
  zip_ref.extractall("/content")
file_path = "/content/denue_inegi_56_clean.csv"
df_csv = pd.read_csv(file_path, encoding= "ISO-8859-1", low_memory= False)

# Coordenadas de apoyo al mapa
repo_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json' #Archivo GeoJSON
mx_regions_geo = requests.get(repo_url).json()

# Hoja de estilos
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Inicializando la app
app = dash.Dash(__name__, external_stylesheets= external_stylesheets)


# Apoyos filtros
# subsector para un selector de RadioItems
available_subsector = df_csv['subsector_scian'].unique()
available_subsector = np.append(available_subsector, 'Todos')


# Modificano el layout de la app
# Inicia contenido del Div principal
app.layout = html.Div([
    # Título H1
    html.H1(children= 'Servicios de Apoyo a los negocios y manejo de desechos y servicios de remediación',
            style= {'textAlign': 'center', 'fontSize': '35px', 'color': '#F5F8FD'}),

    # Primer Div etiquetas, RadioItems, Dropdown
    html.Div([
        
        # Sub Div para títulos
        html.Div([        
            # 'sub-sub' Div 1 para títulos
            html.Div([
                html.H2(children= 'Subsector SCIAN:',
                        style= {'fontSize': '16px', 'marginLeft': '3px','marginTop': '0px', 'color': '#F5F8FD'}),
                html.H2(children= 'Personas Ocupadas:',
                        style= {'fontSize': '16px', 'marginLeft': '3px','marginTop': '-6px', 'color': '#F5F8FD'})

            ], style= {'width': '15%', 'display': 'inline-block'}), # anchura 48%, primera mitad, display uno a lado del otro

            # 'sub-sub' Div 2 para RadioItems y Dropdown
            html.Div([
                # RadioItems
                dcc.RadioItems(
                    id= 'subsector-selection',
                    options= [{'label': i, 'value': i} for i in available_subsector],
                    value= 'Todos',
                    labelStyle = {'display': 'inline-block'},
                    style={'color': '#F5F8FD'}
                ),
                # Dropdown con información de indicadores (lista de opciones)
                dcc.Dropdown(
                    id = 'rama-selection',
                    # Opciones a desplegar
                    #options = [{'label': i, 'value': i} for i in available_rama],
                    # value: valor estalecido de inicio
                    #value = 'Todos'
                )
            ], style= {'width': '84%', 'float': 'right', 'display': 'inline-block'}), # anchura 48%, primera mitad, display uno a lado del otro
        ])
    ], style= {'width': '69%', 'display': 'inline-block'}),


    # Segundo Div. Etiquetas cantidad, porcentaje
    html.Div([
        html.H2(children= 'Negocios (miles)',
                style= {'fontSize': '25px',
                        'textAlign': 'left',
                        'marginTop': '-5px',
                        'marginLeft': '25px',
                        'color': '#F5F8FD'}),
        html.H2(children= 'Negocios proporción',
                style= {'fontSize': '25px',
                        'textAlign': 'left',
                        'marginTop': '-5px',
                        'marginLeft': '25px',
                        'color': '#F5F8FD'})
        
    ], style= {'width': '19%', 'float': 'center', 'display': 'inline-block'}), # anchura 48%, primera mitad, display uno a lado del otro

    # Tercer Div. Cantidad, porcentaje
    html.Div([
        html.H2(id = 'subsector-cantidad',
                style= {'fontSize': '25px',
                        'textAlign': 'left',
                        'marginTop': '-5px',
                        'color': '#F5F8FD'}),
        html.H2(id = 'subsector-porcentaje',
                style= {'fontSize': '25px',
                        'textAlign': 'left',
                        'marginTop': '-5px',
                        'color': '#F5F8FD'})
        
    ], style= {'width': '12%', 'float': 'right', 'display': 'inline-block'}), # anchura 48%, primera mitad, display uno a lado del otro    


    # Cuarto Div. Slider
    html.Div([
        dcc.Slider(
            id= 'year-slider',
            min= df_csv['year'].min(),
            max= df_csv['year'].max(),
            value= df_csv['year'].min(),
            marks= {str(year): str(year) for year in df_csv['year'].unique()},
            step= None
        )
    ], style= {'width': '49%'}),
    
    
    # Quinto Div. Contenedor Graph, para gáficas
    html.Div([
        dcc.Graph(id= 'hist-graphic'),
        dcc.Graph(id= 'treemap-per-ocu')

    ], style= {'width': '49%', 'display': 'inline-block', 'background-color': '#DBE4ED'}),

    # Sexto Div Mapa y treemap
    html.Div([
        dcc.Graph(id= 'map-graphic'),
        dcc.Graph(id= 'serie-time')
    ], style= {'width': '49%', 'display': 'inline-block', 'float': 'right'})

], style = {'background-color': '#043C76', 'padding': '10px 20px 12px 10px'}) #background color de fondo


# ¡Importante! Orden de los input debe ser el mismo para los parámetros de la función

# Dropdown actualización options
@app.callback(
    Output('rama-selection', 'options'),
    Input('subsector-selection', 'value')
)
def rama_options(subsector):
  if subsector != 'Todos':
    dff = df_csv[df_csv['subsector_scian'] == subsector]
    available_rama = dff['rama_scian'].unique()
    available_rama = np.append('Todos', available_rama)
  else:
    available_rama = df_csv['rama_scian'].unique()
    available_rama = np.append('Todos', available_rama)
  return [{'label': i, 'value': i} for i in available_rama]

# Dropdown actualización value
@app.callback(
    Output('rama-selection', 'value'),
    Input('rama-selection', 'options')
)
def rama_value(available_options):
  return available_options[0]['value']


# Leyenda cantidad y proporción
@app.callback(
    Output('subsector-cantidad', 'children'),
    Input('subsector-selection', 'value'),
    Input('rama-selection', 'value')
)
# Actualización cantidad
def update_cantidad(subsector, rama):
  df_proporcion = df_csv.groupby(['subsector_scian', 'rama_scian'])['nom_estab'].count().reset_index()
  total = df_proporcion['nom_estab'].sum()
  
  if subsector != 'Todos' and rama == 'Todos':
    dff = df_proporcion.groupby(['subsector_scian'])['nom_estab'].sum().reset_index()
    total_subsector = dff[dff['subsector_scian'] == subsector].iloc[0,1]
  # si subsector != 'Todos' o == Todos Y rama!= 'Todos, sólo se filtra por rama
  elif rama != 'Todos':
    total_subsector = df_proporcion[df_proporcion['rama_scian'] == rama].iloc[0,2]
  else:
    total_subsector = total
  
  porcentaje = round(total_subsector/1000, 2)
  return f'{str(porcentaje)}'

# Actualización Porcentaje
@app.callback(
    Output('subsector-porcentaje', 'children'),
    Input('subsector-selection', 'value'),
    Input('rama-selection', 'value')
)
def update_percent(subsector, rama):
  df_proporcion = df_csv.groupby(['subsector_scian', 'rama_scian'])['nom_estab'].count().reset_index()
  total = df_proporcion['nom_estab'].sum()
  
  if subsector != 'Todos' and rama == 'Todos':
    dff = df_proporcion.groupby(['subsector_scian'])['nom_estab'].sum().reset_index()
    total_subsector = dff[dff['subsector_scian'] == subsector].iloc[0,1]
  # si subsector != 'Todos' o == Todos Y rama!= 'Todos, sólo se filtra por rama
  elif rama != 'Todos':
    total_subsector = df_proporcion[df_proporcion['rama_scian'] == rama].iloc[0,2]
  else:
    total_subsector = total
  
  porcentaje = round(total_subsector/total*100)
  return f'{str(porcentaje)}%'


# Histograma
@app.callback(
    Output('hist-graphic', 'figure'),
    Input('subsector-selection', 'value'),
    Input('rama-selection', 'value'),
    Input('year-slider', 'value')
)
def update_graph(subsector,
                 rama,
                 year_value):
  # Filtro por año
  dff = df_csv[df_csv['year'] == int(year_value)]

  # Condicional filtros RadioItems, Dropdown
  if subsector != 'Todos' and rama != 'Todos':
    dff = dff[dff['subsector_scian'] == subsector]
    fig = px.histogram(dff[dff['rama_scian'] == rama], x= 'entidad_ok')
  elif subsector == 'Todos' and rama != 'Todos':
    fig = px.histogram(dff[dff['rama_scian'] == rama], x= 'entidad_ok')
  elif subsector != 'Todos' and rama == 'Todos':
    dff = dff[dff['subsector_scian'] == subsector]
    fig = px.histogram(dff, x= 'entidad_ok')
  else:
    fig = px.histogram(dff, x= 'entidad_ok')

  # Actualización de layout de gráfico con datos filtrados
  fig.update_layout(transition_duration= 500)
  # Márgenes (en pixeles; left, bottom, top, right)
  fig.update_layout(height= 400, margin= {'l': 40, 'b': 40, 't': 10, 'r': 10})
  fig.add_annotation(x= 0, y= 0.95, xanchor= 'left', yanchor= 'bottom',
                     xref= 'paper', yref= 'paper', showarrow= False, align= 'left',
                     text= '<b>Negocios por Entidad<b>')
  fig.update_xaxes(title= 'Entidad', showgrid= False) # Eje x
  fig.update_yaxes(title= 'Negocios', showgrid= False) # Eje y
 
  return fig


# Serie de tiempo
@app.callback(
    Output('serie-time', 'figure'),
    Input('subsector-selection', 'value'),
)
def serie_time(subsector):  
  # Condicional por filtro RadioItems
  if subsector != 'Todos':
    dff = df_csv.groupby(['year','subsector_scian'])['nom_estab'].count().reset_index()
    dff = dff[dff['subsector_scian'] == subsector]
  else:
    dff = df_csv.groupby(['year'])['nom_estab'].count().reset_index()

  # Gráfica
  fig = px.line(dff, x= 'year', y= 'nom_estab')
  
  # Actualización de layout de gráfico con datos filtrados
  fig.update_traces(mode= 'lines+markers')
  fig.update_layout(height= 225, margin= {'l': 20, 'b': 30, 'r': 10, 't': 10})
  fig.update_xaxes(title= 'Año', showgrid= False) # Eje x
  fig.update_yaxes(title= 'Negocios', showgrid= False) # Eje y
  
  return fig


# Mapa
@app.callback(
    Output('map-graphic', 'figure'),
    Input('subsector-selection', 'value'),
    Input('rama-selection', 'value'),
    Input('year-slider', 'value')
)
def update_map(subsector, rama, year_value):
  dff = df_csv[df_csv['year'] == int(year_value)]
  # Condicional filtros RadioItems, Dropdown
  if subsector != 'Todos' and rama != 'Todos':
    dff = dff[dff['subsector_scian'] == subsector]
    dff = dff[dff['rama_scian'] == rama]

  elif subsector == 'Todos' and rama != 'Todos':
    dff = dff[dff['rama_scian'] == rama]

  elif subsector != 'Todos' and rama == 'Todos':
    dff = dff[dff['subsector_scian'] == subsector]

  dff = dff.groupby('entidad_ok')['nom_estab'].count().reset_index()
  fig = px.choropleth_mapbox(dff, 
                             geojson=mx_regions_geo, 
                             locations='entidad_ok', # nombre de la columna del Dataframe
                             featureidkey='properties.name',  # ruta al campo del archivo GeoJSON con el que se hará la relación (nombre de los estados)
                             color='nom_estab', #El color depende de las cantidades
                             color_continuous_scale= 'algae', #"burg", #greens
                             mapbox_style="carto-positron",
                             zoom= 3.5,
                             center={"lat": 23.634501, "lon": -102.552784},
                             labels= {'nom_estab': 'Negocios'}
                             )
  fig.update_layout(height= 400, margin={"r":0,"t":0,"l":0,"b":0})
  
  return fig


# Treemap
@app.callback(
    Output('treemap-per-ocu', 'figure'),
    Input('subsector-selection', 'value'),
    Input('rama-selection', 'value'),
    Input('year-slider', 'value')
)
def update_treemap(subsector, rama, year_value):
  dff = df_csv[df_csv['year'] == int(year_value)]
  # Condicional filtros RadioItems, Dropdown
  if subsector != 'Todos' and rama != 'Todos':
    dff = dff[dff['subsector_scian'] == subsector]
    dff = dff[dff['rama_scian'] == rama]

  elif subsector == 'Todos' and rama != 'Todos':
    dff = dff[dff['rama_scian'] == rama]

  elif subsector != 'Todos' and rama == 'Todos':
    dff = dff[dff['subsector_scian'] == subsector]
  
  dff = dff.groupby('per_ocu')['nom_estab'].count().reset_index()
  fig = px.treemap(dff, path= ['per_ocu'], values= 'nom_estab')
  fig.update_layout(height= 225, margin= {'l': 20, 'b': 30, 'r': 10, 't': 10})
  fig.add_annotation(x= 0.005, y= 0.90, xanchor= 'left', yanchor= 'bottom',
                     xref= 'paper', yref= 'paper', showarrow= False, align= 'left',
                     text= '<b>Distribución de negocios por personas ocupadas<b>') 
  return fig


# Servidor
if __name__ == '__main__':
  app.run_server(debug= True)

Overwriting my_dash_app1.py


In [15]:
# Ejecutar la app
tunnel_start()
# Debe ejecutarse desde python como una app en la máquina virtual
!python my_dash_app1.py 

Podes acceder desde https://a89e-35-243-197-117.ngrok.io
my_dash_app1.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
my_dash_app1.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "my_dash_app1" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
/content/my_dash_app1.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/content/my_dash_app1.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from das